In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, classification_report, RocCurveDisplay, roc_curve, auc
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_class_weight

from confidence_intervals import evaluate_with_conf_int
from confidence_intervals.utils import barplot_with_ci

import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib

In [2]:
def claculate_sensitivity(y_test, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred, labels=['Benign', 'Malignant']).ravel()
    sensitivity = tp/(tp+fn)
    return sensitivity

In [3]:
def claculate_specificity(y_test, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred, labels=['Benign', 'Malignant']).ravel()
    specificity = tn / (tn+fp)
    return specificity

In [4]:
def round_to_3dp(d):
    if isinstance(d, float):
        return round(d, 3)
    elif isinstance(d, tuple):
        return tuple(round_to_3dp(i) for i in d)
    elif isinstance(d, dict):
        return {k: round_to_3dp(v) for k, v in d.items()}
    return d

In [5]:
feature_sets = ['FeatureStates', 'OpenSmile', 'MFCC']

In [6]:
models = ['SVM', 'NN', 'LR']

In [7]:
cwd = os.getcwd()

In [8]:
parent_dir = os.path.dirname(cwd)
grandparent_dir = os.path.dirname(parent_dir)

In [9]:
test_files = joblib.load(f'{grandparent_dir}/FEMH_test_files.pkl')

In [10]:
alpha = 5 
num_bootstraps = int(50/alpha*100)

In [11]:
demographics = pd.read_pickle(f'{grandparent_dir}/Audio/medicalhistory.pkl')
demographics = demographics.drop(['ID', 'Disease category', 'pathology'], axis=1)
demographics.head()

,Sex,Age,Narrow pitch range,Decreased volume,Fatigue,Dryness,Lumping,Heartburn,Choking,Eye dryness,...,Noise at work,Occupational vocal demand,Diabetes,Hypertension,CAD,Head and Neck Cancer,Head injury,CVA,Voice handicap index - 10,filename
0,1,97,0,0,0,0,0,0,1,0,...,1,2,0,0,0,0,0,0,12,Atrophy-00002mg
1,1,86,0,0,0,0,0,0,1,0,...,1,4,0,1,0,0,0,1,36,Atrophy-0001297
2,2,45,0,0,0,1,0,0,0,0,...,1,3,0,0,0,0,0,0,16,Atrophy-0001apo
3,1,75,1,1,0,0,1,0,0,0,...,1,3,0,0,1,0,0,0,19,Atrophy-0001qd3
4,1,64,0,0,0,1,0,0,0,0,...,1,2,0,0,1,0,0,0,34,Atrophy-0002ipt


In [12]:
columns = ['FEMH (holdout test set)','FEMH (holdout test set)','FEMH (holdout test set)', 'SVD (external test set)', 'SVD (external test set)', 'SVD (external test set)']

In [13]:
columns2 = ['FeatureStates', 'OpenSmile', 'MFCC', 'FeatureStates', 'OpenSmile', 'MFCC']

In [14]:
num_models = len(models)

In [15]:
unique_inputs = ['Voice', 'Voice + Demographics']
num_inputs = len(unique_inputs)

In [16]:
unique_metrics = ['Balanced Accuracy', 'Sensitivity', 'Specificity', 'AUC']
num_metrics = len(unique_metrics)

In [17]:
metrics = unique_metrics*num_models*num_inputs
print(metrics)
print(len(metrics))

['Balanced Accuracy', 'Sensitivity', 'Specificity', 'AUC', 'Balanced Accuracy', 'Sensitivity', 'Specificity', 'AUC', 'Balanced Accuracy', 'Sensitivity', 'Specificity', 'AUC', 'Balanced Accuracy', 'Sensitivity', 'Specificity', 'AUC', 'Balanced Accuracy', 'Sensitivity', 'Specificity', 'AUC', 'Balanced Accuracy', 'Sensitivity', 'Specificity', 'AUC']
24


In [18]:
inputs = [x for x in unique_inputs for _ in range(num_metrics)]*num_models
print(inputs)
print(len(inputs))

['Voice', 'Voice', 'Voice', 'Voice', 'Voice + Demographics', 'Voice + Demographics', 'Voice + Demographics', 'Voice + Demographics', 'Voice', 'Voice', 'Voice', 'Voice', 'Voice + Demographics', 'Voice + Demographics', 'Voice + Demographics', 'Voice + Demographics', 'Voice', 'Voice', 'Voice', 'Voice', 'Voice + Demographics', 'Voice + Demographics', 'Voice + Demographics', 'Voice + Demographics']
24


In [19]:
classifiers = [x for x in models for _ in range(num_metrics*num_inputs)]
print(classifiers)
print(len(classifiers))

['SVM', 'SVM', 'SVM', 'SVM', 'SVM', 'SVM', 'SVM', 'SVM', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'LR', 'LR', 'LR', 'LR', 'LR', 'LR', 'LR', 'LR']
24


In [20]:
df = pd.DataFrame(columns = [columns, columns2], index=[classifiers, inputs, metrics])
df

FEMH (holdout test set)            \
                                                     FeatureStates OpenSmile   
SVM Voice                Balanced Accuracy                     NaN       NaN   
                         Sensitivity                           NaN       NaN   
                         Specificity                           NaN       NaN   
                         AUC                                   NaN       NaN   
    Voice + Demographics Balanced Accuracy                     NaN       NaN   
                         Sensitivity                           NaN       NaN   
                         Specificity                           NaN       NaN   
                         AUC                                   NaN       NaN   
NN  Voice                Balanced Accuracy                     NaN       NaN   
                         Sensitivity                           NaN       NaN   
                         Specificity                           NaN       NaN   
                         AUC                                   NaN       NaN   
    Voice + Demographics Balanced Accuracy                     NaN       NaN   
                         Sensitivity                           NaN       NaN   
                         Specificity                           NaN       NaN   
                         AUC                                   NaN       NaN   
LR  Voice                Balanced Accuracy                     NaN       NaN   
                         Sensitivity                           NaN       NaN   
                         Specificity                           NaN       NaN   
                         AUC                                   NaN       NaN   
    Voice + Demographics Balanced Accuracy                     NaN       NaN   
                         Sensitivity                           NaN       NaN   
                         Specificity                           NaN       NaN   
                         AUC                                   NaN       NaN   

                                                SVD (external test set)  \
                                           MFCC           FeatureStates   
SVM Voice                Balanced Accuracy  NaN                     NaN   
                         Sensitivity        NaN                     NaN   
                         Specificity        NaN                     NaN   
                         AUC                NaN                     NaN   
    Voice + Demographics Balanced Accuracy  NaN                     NaN   
                         Sensitivity        NaN                     NaN   
                         Specificity        NaN                     NaN   
                         AUC                NaN                     NaN   
NN  Voice                Balanced Accuracy  NaN                     NaN   
                         Sensitivity        NaN                     NaN   
                         Specificity        NaN                     NaN   
                         AUC                NaN                     NaN   
    Voice + Demographics Balanced Accuracy  NaN                     NaN   
                         Sensitivity        NaN                     NaN   
                         Specificity        NaN                     NaN   
                         AUC                NaN                     NaN   
LR  Voice                Balanced Accuracy  NaN                     NaN   
                         Sensitivity        NaN                     NaN   
                         Specificity        NaN                     NaN   
                         AUC                NaN                     NaN   
    Voice + Demographics Balanced Accuracy  NaN                     NaN   
                         Sensitivity        NaN                     NaN   
                         Specificity        NaN                     NaN   
                         AUC                NaN                     NaN   

           

In [21]:
svd_accuracy_data = {}
svd_sensitivity_data = {}
svd_specificity_data = {}
svd_auc_data = {}

for model in models:
    print('*'*10)
    print(model)
    
    for feature in feature_sets:
        print(feature)
        if feature == 'MFCC':
            test_df = pd.read_pickle(f"{grandparent_dir}/Raw Features/{feature}_SVD.pkl")
        else:
            test_df = pd.read_csv(f"{grandparent_dir}/Raw Features/{feature}_SVD.csv", index_col=0)
        
        classifiers = [f'{model}_{feature}_Rec_Only', f'{model}_{feature}_Age_Sex']
        # classifiers = [f'SVM_{feature}_Rec_Only']
        
        classifier_labels = {f'{model}_{feature}_Rec_Only':'Voice',
                        f'{model}_{feature}_Age_Sex':'Voice + Demographics'
                        }
        
        for classifier in classifiers:
            print(classifier_labels[classifier])
            best_pipeline = joblib.load(f'Models/{classifier}.pkl')
            features = best_pipeline.feature_names_in_
    
            # Prepare the features and labels
            X_test = test_df[features]
            y_test = test_df['pathology']
    
            y_pred = best_pipeline.predict(X_test)
            y_pred_prob = best_pipeline.predict_proba(X_test)[:, 1] 

            df.at[(model, classifier_labels[classifier], 'Balanced Accuracy'), ('SVD (external test set)', feature)] = round_to_3dp(evaluate_with_conf_int(y_pred, balanced_accuracy_score, y_test, num_bootstraps=num_bootstraps, alpha=alpha))
            df.at[(model, classifier_labels[classifier], 'AUC'), ('SVD (external test set)', feature)] = round_to_3dp(evaluate_with_conf_int(y_pred_prob, roc_auc_score, y_test, num_bootstraps=num_bootstraps, alpha=alpha))
            df.at[(model, classifier_labels[classifier], 'Sensitivity'), ('SVD (external test set)', feature)] = round_to_3dp(evaluate_with_conf_int(y_pred, claculate_sensitivity, y_test, num_bootstraps=num_bootstraps, alpha=alpha))
            df.at[(model, classifier_labels[classifier], 'Specificity'), ('SVD (external test set)', feature)] = round_to_3dp(evaluate_with_conf_int(y_pred, claculate_specificity, y_test, num_bootstraps=num_bootstraps, alpha=alpha))

**********
SVM
FeatureStates
Voice
Voice + Demographics
OpenSmile
Voice
Voice + Demographics
MFCC
Voice
Voice + Demographics
**********
NN
FeatureStates
Voice
Voice + Demographics
OpenSmile
Voice
Voice + Demographics
MFCC
Voice
Voice + Demographics
**********
LR
FeatureStates
Voice
Voice + Demographics
OpenSmile
Voice
Voice + Demographics
MFCC
Voice
Voice + Demographics


In [22]:
df

FEMH (holdout test set)            \
                                                     FeatureStates OpenSmile   
SVM Voice                Balanced Accuracy                     NaN       NaN   
                         Sensitivity                           NaN       NaN   
                         Specificity                           NaN       NaN   
                         AUC                                   NaN       NaN   
    Voice + Demographics Balanced Accuracy                     NaN       NaN   
                         Sensitivity                           NaN       NaN   
                         Specificity                           NaN       NaN   
                         AUC                                   NaN       NaN   
NN  Voice                Balanced Accuracy                     NaN       NaN   
                         Sensitivity                           NaN       NaN   
                         Specificity                           NaN       NaN   
                         AUC                                   NaN       NaN   
    Voice + Demographics Balanced Accuracy                     NaN       NaN   
                         Sensitivity                           NaN       NaN   
                         Specificity                           NaN       NaN   
                         AUC                                   NaN       NaN   
LR  Voice                Balanced Accuracy                     NaN       NaN   
                         Sensitivity                           NaN       NaN   
                         Specificity                           NaN       NaN   
                         AUC                                   NaN       NaN   
    Voice + Demographics Balanced Accuracy                     NaN       NaN   
                         Sensitivity                           NaN       NaN   
                         Specificity                           NaN       NaN   
                         AUC                                   NaN       NaN   

                                                 SVD (external test set)  \
                                           MFCC            FeatureStates   
SVM Voice                Balanced Accuracy  NaN  (0.628, (0.557, 0.696))   
                         Sensitivity        NaN    (0.763, (0.621, 0.9))   
                         Specificity        NaN  (0.493, (0.466, 0.521))   
                         AUC                NaN  (0.649, (0.562, 0.735))   
    Voice + Demographics Balanced Accuracy  NaN  (0.609, (0.529, 0.691))   
                         Sensitivity        NaN  (0.526, (0.364, 0.688))   
                         Specificity        NaN  (0.691, (0.666, 0.717))   
                         AUC                NaN  (0.716, (0.642, 0.786))   
NN  Voice                Balanced Accuracy  NaN  (0.628, (0.547, 0.705))   
                         Sensitivity        NaN  (0.526, (0.368, 0.676))   
                         Specificity        NaN  (0.729, (0.706, 0.753))   
                         AUC                NaN  (0.692, (0.607, 0.772))   
    Voice + Demographics Balanced Accuracy  NaN  (0.669, (0.586, 0.748))   
                         Sensitivity        NaN   (0.553, (0.39, 0.704))   
                         Specificity        NaN  (0.786, (0.764, 0.808))   
                         AUC                NaN  (0.715, (0.627, 0.789))   
LR  Voice                Balanced Accuracy  NaN  (0.619, (0.541, 0.689))   
                         Sensitivity        NaN  (0.711, (0.558, 0.849))   
                         Specificity        NaN  (0.528, (0.502, 0.557))   
                         AUC                NaN  (0.665, (0.578, 0.747))   
    Voice + Demographics Balanced Accuracy  NaN  (0.661, (0.586, 0.735))   
                         Sensitivity        NaN  (0.711, (0.558, 0.849))   
                         Specificity        NaN  (0.612, (0.587, 0.639))   
                         AUC                NaN  (0.687, (0.60

In [23]:
femh_accuracy_data = {}
femh_sensitivity_data = {}
femh_specificity_data = {}
femh_auc_data = {}

for model in models:
    print('*'*10)
    print(model)
    for feature in feature_sets:
        print(feature)
        classifiers = [f'{model}_{feature}_Rec_Only', f'{model}_{feature}_Age_Sex']
    
        classifier_labels = {f'{model}_{feature}_Rec_Only':'Voice',
                        f'{model}_{feature}_Age_Sex':'Voice + Demographics'
                        }

        if feature == 'MFCC':
            femh_df = pd.read_pickle(f"{grandparent_dir}/Raw Features/{feature}_FEMH.pkl")
        else:
            femh_df = pd.read_csv(f"{grandparent_dir}/Raw Features/{feature}_FEMH.csv", index_col=0)

        femh_df['filename'] = femh_df['file'].str.split('.', expand=True)[0]
        femh_df = femh_df.drop(['file'], axis=1)

        femh_df = pd.merge(femh_df, demographics, on='filename', how='inner')
        
        # Pathologies to be replaced with "Malignant"
        malignant_pathologies = ['Laryngeal cancer', 'Dysplasia']
        
        # Replace specified pathologies with "Malignant"
        femh_df['pathology'] = femh_df['pathology'].apply(lambda x: 'Malignant' if x in malignant_pathologies else 'Benign')

        test_df = femh_df[femh_df['filename'].isin(test_files)]
        test_df = test_df.reset_index(drop=True)

        for classifier in classifiers:
            print(classifier_labels[classifier])
            best_pipeline = joblib.load(f'Models/{classifier}.pkl')
            features = best_pipeline.feature_names_in_
    
            # Prepare the features and labels
            X_test = test_df[features]
            y_test = test_df['pathology']
    
            y_pred = best_pipeline.predict(X_test)
            y_pred_prob = best_pipeline.predict_proba(X_test)[:, 1] 
    
            if classifier_labels[classifier] not in femh_accuracy_data:
                femh_accuracy_data[classifier_labels[classifier]] = {}
                femh_auc_data[classifier_labels[classifier]] = {}
                femh_sensitivity_data[classifier_labels[classifier]] = {}
                femh_specificity_data[classifier_labels[classifier]] = {}
            
            df.at[(model, classifier_labels[classifier], 'Balanced Accuracy'), ('FEMH (holdout test set)', feature)] = round_to_3dp(evaluate_with_conf_int(y_pred, balanced_accuracy_score, y_test, num_bootstraps=num_bootstraps, alpha=alpha))
            df.at[(model, classifier_labels[classifier], 'AUC'), ('FEMH (holdout test set)', feature)] = round_to_3dp(evaluate_with_conf_int(y_pred_prob, roc_auc_score, y_test, num_bootstraps=num_bootstraps, alpha=alpha))
            df.at[(model, classifier_labels[classifier], 'Sensitivity'), ('FEMH (holdout test set)', feature)] = round_to_3dp(evaluate_with_conf_int(y_pred, claculate_sensitivity, y_test, num_bootstraps=num_bootstraps, alpha=alpha))
            df.at[(model, classifier_labels[classifier], 'Specificity'), ('FEMH (holdout test set)', feature)] = round_to_3dp(evaluate_with_conf_int(y_pred, claculate_specificity, y_test, num_bootstraps=num_bootstraps, alpha=alpha))

**********
SVM
FeatureStates
Voice
Voice + Demographics
OpenSmile
Voice
Voice + Demographics
MFCC
Voice
Voice + Demographics
**********
NN
FeatureStates
Voice
Voice + Demographics
OpenSmile
Voice
Voice + Demographics
MFCC
Voice
Voice + Demographics
**********
LR
FeatureStates
Voice
Voice + Demographics
OpenSmile
Voice
Voice + Demographics
MFCC
Voice
Voice + Demographics


In [24]:
df.to_csv('ClassificationResultsVoiceDemographics.csv')

In [25]:
import ast

In [26]:
df = pd.read_csv('ClassificationResultsVoiceDemographics.csv', index_col=[0,1,2], header=[0,1])

In [27]:
df.columns

MultiIndex([('FEMH (holdout test set)', 'FeatureStates'),
            ('FEMH (holdout test set)',     'OpenSmile'),
            ('FEMH (holdout test set)',          'MFCC'),
            ('SVD (external test set)', 'FeatureStates'),
            ('SVD (external test set)',     'OpenSmile'),
            ('SVD (external test set)',          'MFCC')],
           )

In [28]:
for col in df.columns:
    print(col)
    df[col] = df[col].apply(ast.literal_eval)

('FEMH (holdout test set)', 'FeatureStates')
('FEMH (holdout test set)', 'OpenSmile')
('FEMH (holdout test set)', 'MFCC')
('SVD (external test set)', 'FeatureStates')
('SVD (external test set)', 'OpenSmile')
('SVD (external test set)', 'MFCC')


In [29]:
df_formatted = df.copy()

In [30]:
df_formatted

FEMH (holdout test set)  \
                                                      FeatureStates   
SVM Voice                Balanced Accuracy  (0.691, (0.593, 0.784))   
                         Sensitivity          (0.68, (0.48, 0.857))   
                         Specificity        (0.702, (0.665, 0.736))   
                         AUC                 (0.76, (0.665, 0.845))   
    Voice + Demographics Balanced Accuracy    (0.7, (0.594, 0.789))   
                         Sensitivity         (0.64, (0.429, 0.818))   
                         Specificity        (0.759, (0.726, 0.793))   
                         AUC                (0.773, (0.682, 0.855))   
NN  Voice                Balanced Accuracy  (0.683, (0.576, 0.784))   
                         Sensitivity         (0.52, (0.318, 0.714))   
                         Specificity        (0.846, (0.817, 0.873))   
                         AUC                (0.754, (0.643, 0.857))   
    Voice + Demographics Balanced Accuracy   (0.629, (0.522, 0.73))   
                         Sensitivity          (0.44, (0.227, 0.64))   
                         Specificity        (0.819, (0.786, 0.847))   
                         AUC                (0.758, (0.663, 0.833))   
LR  Voice                Balanced Accuracy   (0.653, (0.551, 0.75))   
                         Sensitivity          (0.56, (0.36, 0.762))   
                         Specificity        (0.746, (0.713, 0.779))   
                         AUC                (0.724, (0.604, 0.834))   
    Voice + Demographics Balanced Accuracy  (0.657, (0.544, 0.756))   
                         Sensitivity          (0.56, (0.345, 0.75))   
                         Specificity        (0.754, (0.722, 0.787))   
                         AUC                (0.733, (0.627, 0.827))   

                                                                     \
                                                          OpenSmile   
SVM Voice                Balanced Accuracy    (0.68, (0.58, 0.772))   
                         Sensitivity          (0.64, (0.44, 0.818))   
                         Specificity         (0.72, (0.686, 0.754))   
                         AUC                (0.742, (0.676, 0.805))   
    Voice + Demographics Balanced Accuracy  (0.752, (0.656, 0.837))   
                         Sensitivity         (0.76, (0.571, 0.923))   
                         Specificity        (0.743, (0.709, 0.776))   
                         AUC                (0.802, (0.743, 0.851))   
NN  Voice                Balanced Accuracy  (0.632, (0.533, 0.727))   
                         Sensitivity           (0.56, (0.36, 0.75))   
                         Specificity         (0.704, (0.668, 0.74))   
                         AUC                (0.709, (0.637, 0.779))   
    Voice + Demographics Balanced Accuracy  (0.755, (0.659, 0.841))   
                         Sensitivity         (0.76, (0.571, 0.923))   
                         Specificity         (0.75, (0.717, 0.784))   
                         AUC                (0.809, (0.753, 0.858))   
LR  Voice                Balanced Accuracy  (0.663, (0.555, 0.761))   
                         Sensitivity         (0.64, (0.429, 0.833))   
                         Specificity        (0.685, (0.649, 0.721))   
                         AUC                 (0.714, (0.637, 0.79))   
    Voice + Demographics Balanced Accuracy  (0.797, (0.722, 0.859))   
                         Sensitivity           (0.88, (0.733, 1.0))   
                         Specificity        (0.715, (0.677, 0.749))   
                         AUC                (0.836, (0.767, 0.892))   

                                                                     \
                                                               MFCC   
SVM Voice                Balanced Accuracy  (0.631, (0.531, 0.731))   
                         Sensitivity          (0.4, (0.206, 0.609))   
                         Specificity        (0.861, (0.835, 0.887)

In [31]:
def format_nested(tup):
    formatted = []
    for val in tup:
        if isinstance(val, tuple):
            # If it's a nested tuple, recursively format it
            formatted.append(format_nested(val))
        else:
            # If it's a number, format it to 3 decimal places
            formatted.append(format(val, '.3f'))
    return tuple(formatted)

In [32]:
df_formatted = df_formatted.applymap(format_nested)

C:\Users\maryp\AppData\Local\Temp\ipykernel_13344\3789772394.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_formatted = df_formatted.applymap(format_nested)


In [33]:
type(df_formatted.iloc[0][col])

tuple

In [34]:
for col in df_formatted.columns.levels[0]:  # Iterate over first level of columns
    print('*'*10)
    print(col)
    
    for row in df_formatted.index:
        print(row)
        
        # Access all sub-columns under the current top-level column (col)
        subcols = df_formatted[col].columns
        
        # Apply the lambda function over all sub-columns
        max_val = df_formatted[col].loc[row].apply(lambda x: x[0]).max()  # Find the maximum value
        print(max_val)
        
        # Now update values based on the condition
        df_formatted[col].loc[row] = df_formatted[col].loc[row].apply(
            lambda x: f"\\makecell{{\\textbf{{{x[0]}}} \\ \\ ({x[1][0]}, {x[1][1]})}}"
            if x[0] == max_val 
            else f"\\makecell{{{x[0]} \\\\ ({x[1][0]}, {x[1][1]})}}"
        )


**********
FEMH (holdout test set)
('SVM', 'Voice', 'Balanced Accuracy')
0.691
('SVM', 'Voice', 'Sensitivity')
0.680
('SVM', 'Voice', 'Specificity')
0.861
('SVM', 'Voice', 'AUC')
0.760
('SVM', 'Voice + Demographics', 'Balanced Accuracy')
0.752
('SVM', 'Voice + Demographics', 'Sensitivity')
0.760
('SVM', 'Voice + Demographics', 'Specificity')
0.802
('SVM', 'Voice + Demographics', 'AUC')
0.807
('NN', 'Voice', 'Balanced Accuracy')
0.683
('NN', 'Voice', 'Sensitivity')
0.560
('NN', 'Voice', 'Specificity')
0.846
('NN', 'Voice', 'AUC')
0.754
('NN', 'Voice + Demographics', 'Balanced Accuracy')
0.755
('NN', 'Voice + Demographics', 'Sensitivity')
0.760
('NN', 'Voice + Demographics', 'Specificity')
0.871
('NN', 'Voice + Demographics', 'AUC')
0.809
('LR', 'Voice', 'Balanced Accuracy')
0.666
('LR', 'Voice', 'Sensitivity')
0.640
('LR', 'Voice', 'Specificity')
0.746
('LR', 'Voice', 'AUC')
0.724
('LR', 'Voice + Demographics', 'Balanced Accuracy')
0.797
('LR', 'Voice + Demographics', 'Sensitivity')
0.8

C:\Users\maryp\AppData\Local\Temp\ipykernel_13344\3557638342.py:16: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_formatted[col].loc[row] = df_formatted[col].loc[row].apply(
C:\Users\maryp\AppData\Local\Temp\ipykernel_13344\3557638342.py:

In [35]:
df_formatted

FEMH (holdout test set)  \
                                                                           FeatureStates   
SVM Voice                Balanced Accuracy  \makecell{\textbf{0.691} \ \ (0.593, 0.784)}   
                         Sensitivity        \makecell{\textbf{0.680} \ \ (0.480, 0.857)}   
                         Specificity                  \makecell{0.702 \\ (0.665, 0.736)}   
                         AUC                \makecell{\textbf{0.760} \ \ (0.665, 0.845)}   
    Voice + Demographics Balanced Accuracy            \makecell{0.700 \\ (0.594, 0.789)}   
                         Sensitivity                  \makecell{0.640 \\ (0.429, 0.818)}   
                         Specificity                  \makecell{0.759 \\ (0.726, 0.793)}   
                         AUC                          \makecell{0.773 \\ (0.682, 0.855)}   
NN  Voice                Balanced Accuracy  \makecell{\textbf{0.683} \ \ (0.576, 0.784)}   
                         Sensitivity                  \makecell{0.520 \\ (0.318, 0.714)}   
                         Specificity        \makecell{\textbf{0.846} \ \ (0.817, 0.873)}   
                         AUC                \makecell{\textbf{0.754} \ \ (0.643, 0.857)}   
    Voice + Demographics Balanced Accuracy            \makecell{0.629 \\ (0.522, 0.730)}   
                         Sensitivity                  \makecell{0.440 \\ (0.227, 0.640)}   
                         Specificity                  \makecell{0.819 \\ (0.786, 0.847)}   
                         AUC                          \makecell{0.758 \\ (0.663, 0.833)}   
LR  Voice                Balanced Accuracy            \makecell{0.653 \\ (0.551, 0.750)}   
                         Sensitivity                  \makecell{0.560 \\ (0.360, 0.762)}   
                         Specificity        \makecell{\textbf{0.746} \ \ (0.713, 0.779)}   
                         AUC                \makecell{\textbf{0.724} \ \ (0.604, 0.834)}   
    Voice + Demographics Balanced Accuracy            \makecell{0.657 \\ (0.544, 0.756)}   
                         Sensitivity                  \makecell{0.560 \\ (0.345, 0.750)}   
                         Specificity                  \makecell{0.754 \\ (0.722, 0.787)}   
                         AUC                          \makecell{0.733 \\ (0.627, 0.827)}   

                                                                                          \
                                                                               OpenSmile   
SVM Voice                Balanced Accuracy            \makecell{0.680 \\ (0.580, 0.772)}   
                         Sensitivity                  \makecell{0.640 \\ (0.440, 0.818)}   
                         Specificity                  \makecell{0.720 \\ (0.686, 0.754)}   
                         AUC                          \makecell{0.742 \\ (0.676, 0.805)}   
    Voice + Demographics Balanced Accuracy  \makecell{\textbf{0.752} \ \ (0.656, 0.837)}   
                         Sensitivity        \makecell{\textbf{0.760} \ \ (0.571, 0.923)}   
                         Specificity                  \makecell{0.743 \\ (0.709, 0.776)}   
                         AUC                          \makecell{0.802 \\ (0.743, 0.851)}   
NN  Voice                Balanced Accuracy            \makecell{0.632 \\ (0.533, 0.727)}   
                         Sensitivity        \makecell{\textbf{0.560} \ \ (0.360, 0.750)}   
                         Specificity                  \makecell{0.704 \\ (0.668, 0.740)}   
                         AUC                          \makecell{0.709 \\ (0.637, 0.779)}   
    Voice + Demographics Balanced Accuracy  \makecell{\textbf{0.755} \ \ (0.659, 0.841)}   
                         Sensitivity        \makecell{\textbf{0.760} \ \ (0.571, 0.923)}   
                         Specificity                  \makecell{0.750 \\ (0.717, 0.784)}   
                         AUC                \makecell{\textbf{0.809} \ \ (0.753, 0.858)}   
LR  Voice       

In [36]:
latex_code = df_formatted.to_latex(column_format='x{1.5cm}|x{1.5cm}|c|ccc|ccc', multicolumn_format='c|')

In [37]:
print(latex_code)

\begin{tabular}{x{1.5cm}|x{1.5cm}|c|ccc|ccc}
\toprule
 &  &  & \multicolumn{3}{c|}{FEMH (holdout test set)} & \multicolumn{3}{c|}{SVD (external test set)} \\
 &  &  & FeatureStates & OpenSmile & MFCC & FeatureStates & OpenSmile & MFCC \\
\midrule
\multirow[t]{8}{*}{SVM} & \multirow[t]{4}{*}{Voice} & Balanced Accuracy & \makecell{\textbf{0.691} \ \ (0.593, 0.784)} & \makecell{0.680 \\ (0.580, 0.772)} & \makecell{0.631 \\ (0.531, 0.731)} & \makecell{\textbf{0.628} \ \ (0.557, 0.696)} & \makecell{0.582 \\ (0.497, 0.662)} & \makecell{0.532 \\ (0.462, 0.613)} \\
 &  & Sensitivity & \makecell{\textbf{0.680} \ \ (0.480, 0.857)} & \makecell{0.640 \\ (0.440, 0.818)} & \makecell{0.400 \\ (0.206, 0.609)} & \makecell{\textbf{0.763} \ \ (0.621, 0.900)} & \makecell{0.500 \\ (0.341, 0.667)} & \makecell{0.289 \\ (0.154, 0.455)} \\
 &  & Specificity & \makecell{0.702 \\ (0.665, 0.736)} & \makecell{0.720 \\ (0.686, 0.754)} & \makecell{\textbf{0.861} \ \ (0.835, 0.887)} & \makecell{0.493 \\ (0.466, 0.521